In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1590345673634_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# import further libraries
#from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf,count,when,isnan,regexp_replace,countDistinct,month,from_unixtime,to_timestamp,lead,datediff,mean
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression,RandomForestClassifier, GBTClassifier
from pyspark.ml.feature import MinMaxScaler, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
import datetime
import pyspark.sql.functions as F
import numpy as np
import pandas as pd


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'



In [3]:
spark = SparkSession\
        .builder.appName("SparkifyThomas").getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:

# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
user_log = spark.read.json(event_data)
user_log.persist()
user_log = user_log.filter(user_log["userId"] != "")





VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# ### Churn dataset as in 'data wrangling with DataFrame'
# In[11]:
flag_downgrade_event = udf(lambda x: 1 if x == "Submit Downgrade" else 0, IntegerType())
user_log = user_log.withColumn("downgraded", flag_downgrade_event("page"))
# now we have for each timestamp sorted user a delta peak where the downgrade took place
from pyspark.sql import Window
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
user_log = user_log.withColumn("churn", Fsum("downgraded").over(windowval))




In [ ]:
get_time = udf(lambda x: pd.Timestamp(x / 1000.0,unit='s').strftime("%Y-%m-%d %H:%M:%S"))
user_log = user_log.withColumn("time", get_time(user_log.ts)).withColumn('registration_time',get_time(user_log.registration))
user_log = user_log.withColumn('date', from_unixtime(col('ts')/1000).cast(DateType()))
user_log = user_log.withColumn('month', month("time").alias('month'))
user_log.head()

In [ ]:

churningusers = user_log.groupBy("userId").max("churn").withColumnRenamed("max(churn)", "churn")
churningusers.select(["userId", "churn"])


In [ ]:

#Recoding of the categorical variables....
transformchurn  = udf(lambda x: 1 if x > 0  else 0, IntegerType()) # GaussBost Binary!
transformgender = udf(lambda x: 1 if x == "F" else 0, IntegerType())
transformlevel =  udf(lambda x: 1 if x == "paid" else 0, IntegerType())
user_temp1 = user_log.withColumn("gender", transformgender("gender"))
user_temp2 = user_log.withColumn("level",  transformlevel ("level"))
user_temp3 = user_log.withColumn("churngbt",  transformlevel ("churn"))
user_temp1 = user_temp1.groupby('userId').agg({"gender": "max"}).withColumnRenamed("max(gender)", "gender")
user_temp2 = user_temp2.groupby('userId').agg({"level": "max"}).withColumnRenamed("max(level)", "level")
user_temp3 = user_temp3.groupby('userId').agg({"churngbt": "max"}).withColumnRenamed("max(churngbt)", "churngbt")
churningusers=churningusers.join(user_temp1, ['userId'])
churningusers=churningusers.join(user_temp2, ['userId'])
churningusers=churningusers.join(user_temp3, ['userId'])


In [ ]:
# Finally, join all of them together to the feature set...
# 
# In[100]:
songcount = user_log.groupby("userId").agg(countDistinct("song"))
artistdiversity = user_log.groupby("userId").agg(countDistinct("artist"))
sumoflength=user_log.groupby("userId").agg({"length" : "sum"})
visitfrequency = user_log.groupby("userId").count()
songcount = songcount.withColumnRenamed("count(DISTINCT song)", "songcount")
artistdiversity = artistdiversity.withColumnRenamed("count(DISTINCT artist)", "artistdiversity")
sumoflength=sumoflength.withColumnRenamed("sum(length)","sumoflength")
visitfrequency = visitfrequency.withColumnRenamed("count", "visitfrequency")
first_interaction =  user_log.groupBy('userId').agg(min('ts').alias('first_interaction'))
last_interaction =  user_log.groupBy('userId').agg(max('ts').alias('last_interaction'))
mean_interaction =  user_log.groupBy('userId').agg(mean('ts').alias('mean_interaction'))

In [ ]:
# again the userid with the known churning as a test
churningusers  = churningusers.join(artistdiversity,["userId"])
churningusers  = churningusers.join(visitfrequency,["userId"])
churningusers  = churningusers.join(songcount,["userId"])
churningusers  = churningusers.join(sumoflength,["userId"])
churningusers  = churningusers.join(first_interaction,["userId"])
churningusers  = churningusers.join(last_interaction,["userId"])
churningusers  = churningusers.join(mean_interaction,["userId"])
churningusers = churningusers.withColumn('lastedinteraction', churningusers['last_interaction']-churningusers['first_interaction'])
churningusers = churningusers.withColumn('lastedinteraction', churningusers['lastedinteraction']/1e9)

In [ ]:

churningusers.printSchema()
churningusers.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in churningusers.columns]).collect()

In [ ]:

churningusers= churningusers.drop("userId")
getallfeatures = [mycol for mycol in churningusers.columns if mycol not in ['churn','churngbt']]
print(getallfeatures)
getallfeaturesall = ['gender', 'level', 'artistdiversity', 'visitfrequency', 'songcount', 'sumoflength', 'first_interaction', 'last_interaction', 'mean_interaction', 'lastedinteraction']

getallfeatures1 = ['gender', 'level']
getallfeatures2 = ['gender', 'level', 'artistdiversity' ]
getallfeatures3 = ['gender', 'level', 'artistdiversity', 'visitfrequency']
getallfeatures4 = ['gender', 'level', 'artistdiversity', 'visitfrequency', 'songcount']
getallfeatures5 = ['gender', 'level', 'artistdiversity', 'visitfrequency', 'songcount', 'sumoflength']
getallfeatures = getallfeatures1


In [ ]:
#here, userid is no longer needed, as the model is lter on served by user data, without knowlegde of origin

churningusers.printSchema()

In [ ]:

train, test = churningusers.randomSplit([0.7, 0.3], seed=42)
train = train.cache()

In [ ]:
def modelbuilder(myclassifier, myparam, mynumfold,labelcolin):
    """
    Try to setup a pipeline with given Clasifier
    Input:
    myclassifier - one type of vailable classifier classes
    param - built param grid for  model opt search
    Output :
    mymodel - ML pipeline model as transformer...
    """
    assembler = VectorAssembler(inputCols=getallfeatures, outputCol="features")
    scaler = MinMaxScaler(inputCol="features", outputCol="scfeatures")
    pipeline = Pipeline(stages=[assembler, scaler, myclassifier])
    mymodel = CrossValidator(
        estimator=pipeline,
        estimatorParamMaps=myparam,
        evaluator = \
          MulticlassClassificationEvaluator( \
          labelCol=labelcolin, metricName='f1'),
        numFolds=mynumfold,
    )
    return mymodel


In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="churn")


In [ ]:
rndf = RandomForestClassifier(featuresCol="scfeatures", labelCol="churn")
rndf_param = ParamGridBuilder().build()
rndf_model = modelbuilder(rndf, rndf_param,2)


In [ ]:
rndf_fitted_model = rndf_model.fit(train)
rndf_pred = rndf_fitted_model.transform(test)

rndfscore = evaluator.evaluate(rndf_pred, {evaluator.metricName: "f1"})
print("f1 using Random Forest: {}".format(rndfscore))

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="churngbt")


In [ ]:
gaussboost =GBTClassifier(featuresCol="scfeatures", labelCol="churngbt")
gaussboost_param = ParamGridBuilder().build()
gaussboost_model = modelbuilder(gaussboost, gaussboost_param,2,'churngbt')


In [ ]:
gaussboost_f1score = evaluator.evaluate(gaussboost_pred, {evaluator.metricName: "f1"})
print("f1 using Gradient Boosting: {}".format(gaussboost_f1score))